In [1]:
import overpy
import shapefile
import urllib.request
import time
from geojson import Point, LineString, Polygon, MultiPolygon, MultiLineString, MultiPoint

In [2]:
inshp = "area/reg_buff.shp"
sf = shapefile.Reader(inshp)
bbox = sf.bbox

objects = [['natural', ['natural','name','amenity'], ['geological', 'landcover', 'waterway']],
           ['building', ['building', 'name', 'amenity'], ['shop', 'tourism']],
           ['boundary', ['boundary','name','admin_level','place','koatuu','population'], []],
            ['landuse', ['landuse','name','amenity'], ['aeroway', 'leisure', 'tourism']],
            ['highway', ['highway','name','amenity','int_ref','surface'], ['railway', 'trafic_sign','public_transport']],
            ['man_made', ['man_made','name','amenity',], ['power', 'office', 'shop', 'leisure', 'tourism', 'historic']],
            ['place', ['place','name','koatuu','population'], []],
            ['natural', ['natural','name','amenity'], ['geological', 'landcover']]]

In [92]:
def convert_int(x):
    '''
    Attempts to convert variables to integers
    '''
    try:
        return int(x)
    except ValueError:
        return x
    
    
def get_osm(key):
    '''
    requests data from OSM using Overpass API
    input: key word for OSM data type
    output: dictionary {'node' : overpy.Result, 'way': overpy.Result, 'rel': [....]} 
    '''
    api = overpy.Overpass()
    while True:
        try:
            query = lambda rel_type: api.query("""({}({},{},{},{}) ["{}"];
                                        );
                                        (._;>;);
                                        out;""".format(rel_type, bbox[1], bbox[0], bbox[3], bbox[2], key))
            result = dict()
            result['node'] = query('node')
            result['way'] = query('way')
            result['rel'] = []
            relations = query('rel')
            for ids in relations.get_relation_ids():
                result['rel'].append(api.query("""relation({});(._;>;);out;""".format(ids)))
            return result
        except:
            time.sleep(4)                   #time delay for the next query 4 seconds in case of many requests
        
        
def get_params(obj, schema, additional_tags):
    '''
    returns attribute values for each complete object (e.g. node, way or a component-way in a relation)
    '''
    vals = []
    for tag in schema[1:]:   #do not process the first 'type' attribute yet
        val = obj.tags.get(tag, '')
    vals.append(val)
    val0 = obj.tags.get(schema[0])
    if not val0:
        val0 = list(filter(None, [obj.tags.get(atag, '') for atag in additional_tags]))[0]
    if not val:
        val0 = ""
    vals.insert(0, val0)
    params = list(map(convert_int, vals))   #try to convert to int
    return params


def format_coord(val, geom_type):
    '''
    Formats coordinates (latitude and longitude) to the needed format for json
    input: val - list of coordinates
            geom_type - string; a required geometry type
    output: a list
    '''
    if geom_type == Point:
        return val[0]
    elif geom_type == LineString:
        return val
    elif geom_type == Polygon:
        return [val]

### в relations могут быть nodes together with multipolygons. Probably theiy are accidental. Filter them out!!!

In [4]:
def parse_osm(obj_name, schema, additional_tags):
    '''
    main func
    download osm data with overpass api
    '''
    #get all results for this key/tag and expand them with results from additional tags
    res = get_osm(obj_name)
    for atag in additional_tags:
        ares = get_osm(atag)    #{'node' : overpy.Result, 'way': overpy.Result, 'rel': [....]} for each additional tag
        res['node'].expand(ares['node'])
        res['way'].expand(ares['way'])
        res['rel'].extend(ares['rel'])
               
    #parse osm data
    #1) coordinates for each geometry/relation type
    node = dict()
    way = dict()
    rel = dict()
    node['coords'] = [(float(node.lon), float(node.lat)) for node in res['node'].nodes]
    way['coords'] = [[(float(node.lon), float(node.lat)) for node in way.nodes] for way in res['way'].ways]
    rel['coords'] = [[[(float(node.lon), float(node.lat)) for node in way.nodes] for way in rres.ways] for rres in res['rel']]  #has len(rel['coords']) objects; each has len(rel['coords'][i]) ways
    node['params'] = [get_params(node, schema, additional_tags) for node in res['node'].nodes]
    way['params'] = [get_params(way, schema, additional_tags) for way in res['way'].ways]
    rel['params'] = [get_params(rres.relations[0], schema, additional_tags) for rres in res['rel']]  #here rres represents only one relation
        
    if additional_tags:
        for i, acrd in enumerate(additional_coords):
            type_val = additional_tags[i]
            for j, elem in enumerate(acrd):
                amenity_val = additional_result[i].ways[j].tags.get(additional_tags[i], '')
                try:                                     #if there is an object with the same geometry in the original tag, update the values
                    k = coords.index(elem)
                    params[k][0] = type_val
                    params[k][2] = amenity_val
                    acrd.remove(elem)
                except ValueError:                       #if no object with the same geometry found, add the object
                    coords.append(elem)
                    prm_new = list(map(convert_int, [additional_result[i].ways[j].tags.get(f, '') for f in schema]))
                    prm_new[0] = type_val
                    prm_new[2] = amenity_val
                    params.append(prm_new)
                    
    vals = list(zip(coords, params))


    #filter out different geometry types in the result request
    obj_dict = {Point: [[], []], Polygon: [[], []], LineString: [[], []]}
    for val in vals:
        crd_set = []
        for crd in val[0]:
            if crd not in crd_set:
                crd_set.append(crd)
        if len(val[0]) == 1:
            for i in [0,1]:
                obj_dict[Point][i].append(val[i])
        elif len(val[0]) != len(crd_set):
            for i in [0,1]:
                obj_dict[Polygon][i].append(val[i])
        else:
            for i in [0,1]:
                obj_dict[LineString][i].append(val[i])
                
    #export json   
    schema = list(map(lambda x: x[:10], schema)) #shorten field names to 10 characters
    schema[0] = 'type'     #rename the first field to 'type'
    for geom in obj_dict.keys():
        if obj_dict[geom][0]:    #if the geom type has values for this osm tag
            outjson = {"type": "FeatureCollection", "features": []}
            outname = obj_name + '_' + geom.__name__ + '.geojson'
            for i in range(len(obj_dict[geom][0])):
                feat = {'type': 'Feature',
                        'geometry': geom(format_coord(obj_dict[geom][0][i], geom)),    #{'type': k,'coordinates': format_coord(obj_dict[k][0][i], k)},
                        'properties': dict(zip(schema, obj_dict[geom][1][i]))
                }
                outjson['features'].append(feat)
            with open(outname, 'w') as outfile:
                geojson.dump(outjson, outfile)

In [106]:
a = overpy.Overpass()
r = a.query("""(rel({},{},{},{}) ["{}"];
                                        );
                                        (._;>;);
                                        out;""".format(bbox[1], bbox[0], bbox[3], bbox[2], "boundary"))

In [ ]:
for tag in objects:
    parse_osm(tag[0], tag[1], tag[2])